**Machine learning in production**

The objective of this session is to discover a set of tools designed to
use ML models in real-world applications. We’ll see some examples in the
following categories: - rapid prototyping to explore a set of models -
version control for model parameters - frontend development for end
users - reproducible deployment of applications

# Exercice 1 - *Auto-ML*

See <https://autokeras.com> for documentation.

**Installation**

``` sh
pip install tensorflow
pip install git+https://github.com/keras-team/keras-tuner.git
pip install autokeras
```

As an example, we begin by loading the MNIST dataset. We use the already
provided interface inside the Keras library (but any source will be ok).
We use the traditionl evaluation protocol with the provided train/test
split.

In [ ]:
# ! pip install tensorflow
# ! pip install git+https://github.com/keras-team/keras-tuner.git
# ! pip install autokeras

In [1]:
from keras.datasets import mnist

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train_small = X_train[:100]
Y_train_small = Y_train[:100]

In [2]:
import autokeras as ak

We define a simple image classifier.

In [ ]:
clf = ak.ImageClassifier(overwrite=True, max_trials=2)
clf.fit(X_train_small, Y_train_small, epochs=1)

The best model can be evaluated with

In [ ]:
print(clf.evaluate(X_test, Y_test))

It’s simply a Keras model, we can export and visualize the layers.

In [ ]:
model = clf.export_model()
model.summary()

## Question 1

In the previous cells, it was simply a toy example with no real
exploration. Update the parameters to do more experiments.

In [ ]:
clf = ak.ImageClassifier(overwrite=True, max_trials=1)
clf.fit(X_train_small, Y_train_small, epochs=15)

In [ ]:
print(clf.evaluate(X_test, Y_test))

## Question 2

Using the documentation, customize the search space. Since we are
working on MNIST we can restrict ourself to simple models, without
relying on pre-trained models.


In [ ]:
import numpy as np
import keras
from sklearn import tree

In [13]:
input_node = ak.ImageInput()
output_node = ak.ConvBlock()(input_node)
output_node = ak.Normalization()(output_node)
output_node = ak.ConvBlock()(output_node)
output_node = ak.Normalization()(output_node)
output_node = ak.ClassificationHead()(output_node)

auto_model = ak.AutoModel(inputs=input_node, outputs=output_node, overwrite=True, max_trials=10)
auto_model.fit(X_train_small, Y_train_small, epochs=15)

print(auto_model.evaluate(X_test, Y_test))


Trial 10 Complete [00h 00m 06s]
val_loss: 0.0008931308402679861

Best val_loss So Far: 8.493534551234916e-06
Total elapsed time: 00h 00m 53s
Epoch 1/15
4/4 [==============================] - 2s 16ms/step - loss: 2.5039 - accuracy: 0.1200
Epoch 2/15
4/4 [==============================] - 0s 14ms/step - loss: 2.1033 - accuracy: 0.3500
Epoch 3/15
4/4 [==============================] - 0s 13ms/step - loss: 1.7962 - accuracy: 0.4400
Epoch 4/15
4/4 [==============================] - 0s 12ms/step - loss: 1.4226 - accuracy: 0.6100
Epoch 5/15
4/4 [==============================] - 0s 12ms/step - loss: 1.0350 - accuracy: 0.7200
Epoch 6/15
4/4 [==============================] - 0s 13ms/step - loss: 0.6507 - accuracy: 0.8300
Epoch 7/15
4/4 [==============================] - 0s 13ms/step - loss: 0.4778 - accuracy: 0.8400
Epoch 8/15
4/4 [==============================] - 0s 13ms/step - loss: 0.2891 - accuracy: 0.9100
Epoch 9/15
4/4 [==============================] - 0s 12ms/step - loss: 0.1625 - acc

INFO:tensorflow:Assets written to: .\auto_model\best_model\assets


313/313 [==============================] - 2s 5ms/step - loss: 2.5158 - accuracy: 0.5634
[2.5158190727233887, 0.5633999705314636]


## Question 3

Auto-ML is not limited to deep learning. Use *auto-sklearn*
<https://automl.github.io/auto-sklearn> to explore models from
*scikit-learn*.


In [12]:
import sklearn.metrics
import autosklearn.classification

cls = autosklearn.classification.AutoSklearnClassifier()
cls.fit(X_train_small, Y_train_small)
predictions = cls.predict(X_test)   


print("Accuracy score", sklearn.metrics.accuracy_score(Y_test, predictions))

ModuleNotFoundError: No module named 'autosklearn'


## Question 4

Display a leaderboard presenting the results of all the explorations.

# Exercice 2 - *MLOps*

See <https://mlflow.org> for documentation.

**Installation**

``` sh
pip install mlflow
```

Inside a terminal window, launch

``` sh
mlflow ui
```

And with a browser, go to <http://localhost:5000>.

In [3]:
from sklearn.metrics import accuracy_score

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

Let’s train a simple classifier

In [4]:
from sklearn.neighbors import KNeighborsClassifier

params = {
    "neighbors": 3
}

X_train_for_knn = X_train_small.reshape((-1, 28*28))
X_test_for_knn = X_test.reshape((-1, 28*28))

clf = KNeighborsClassifier(n_neighbors=params["neighbors"])
clf.fit(X_train_for_knn, Y_train_small)

signature = infer_signature(X_train_for_knn, clf.predict(X_train_for_knn))

## Question 5

Analyze the content of the variable signature.

In [5]:
signature

inputs: 
  [Tensor('uint8', (-1, 784))]
outputs: 
  [Tensor('uint8', (-1,))]
params: 
  None

Now, let’s register an experiment.

In [7]:
mlflow.set_experiment("DALAS experiments")

mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the metric
    accuracy = accuracy_score(Y_test, clf.predict(X_test_for_knn))
    mlflow.log_metric("accuracy", accuracy)

    # Log the metric
    accuracy = accuracy_score(Y_test, clf.predict(X_test_for_knn))
    mlflow.log_metric("accuracy", accuracy)

    # Some metadata
    mlflow.set_tag("Training Info", "Basic kNN model for MNIST")

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=clf,
        artifact_path="mnist_model",
        signature=signature,
        input_example=X_train_small,
        registered_model_name="simple-mnist",
    )

Registered model 'simple-mnist' already exists. Creating a new version of this model...
2024/04/04 15:49:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: simple-mnist, version 2
Created version '2' of model 'simple-mnist'.


## Question 6

Explore the view in the web interface.



## Question 7

Run a full experimental campaign, with various parameters and various
kind of models.

## Question 8

MLFlow is designed both for experiments and for deployment. Using
<https://mlflow.org/docs/latest/models.html#python-function-python-function>,
load a model stored on the serve and use it for some classification.

# Exercice 3 - *Interactive* *web* *apps*

In most cases, we want to be able to deploy a demo application. Although
it is not made to be production ready, it is useful to have a show room
in order to demonstrate the capabilities of our model.

There are two main libraries to build a web application for a ML
project: - Gradio <https://github.com/gradio-app/gradio> (the backend of
Huggingface Spaces <https://huggingface.co/spaces>) - Streamlit
<https://github.com/streamlit/streamlit>

## Question 9

Using one of these libraries, build a demo for one of the previously
trained models. The model will be loaded through MLFlow.

# Exercice 4 - *Docker*

Docker is a container framework for Linux. A container is lightweight
kind of a virtual machine, allowing to run a lot of containers without
any overhead. It is particularly suited to deploy complex application
without messing with dependencies and package versions.

For research, distributing a container with all the dependencies
installed is a good solution for reproducible research. For industrial
applications, a container with the model and all the required
dependencies can be easily deploy on the production servers.

## Question 10

*(This part cannot be done at the PPTI.)*

Build a container with all the necessary components to run the web
application you made in the previous part.